# Fahrradmessstellen Ergebnisse Radzählungen 2019

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import re

In [2]:
#Datei einlesen mit open, da encoding anders
with open('ErgebnisseFahrradmessstellenBonn2019.csv', 'r', encoding='utf-8', errors='replace') as file:
    df = pd.read_csv(file, sep=';', header=None, low_memory=False)
    
#Sonderzeichen korrigieren und aufräumen
replacement_dict = {re.compile(r'�'): 'ü'}
for pattern, replacement in replacement_dict.items():
    df = df.replace(pattern, replacement, regex=True)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.set_index('Time', inplace=True)
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M')
df = df.drop(columns=['5.03 BN - Nordbrücke (Südseite)', '5.04 BN - Nordbrücke (Nordseite)', 
                      '5.05 BN - Südbrücke (Südseite)', '5.06 BN - Südbrücke (Nordseite)', 
                      '5.01 BN - Kennedybrücke (Nordseite)', '5.02 BN - Kennedybrücke (Südseite) Barometer', '5.10 BN - Brühltalweg'])


column_mapping = {
    '5.07 BN - Estermannufer': 'Estermannufer',
    '5.08 BN - Von-Sandt-Ufer': 'Von-Sandt-Ufer',
    '5.09 BN - Rhenusallee': 'Rhenusallee',
    '5.11 BN - Brühler Straüe': 'Brühler Straße',
    '5.14 BN - Mc Cloy Weg': 'Mc Cloy Weg',
    '5.15 BN - Weg auf Damm Neil': 'Weg auf Damm Neil',
    '5.13 BN - Wilhelm-Spiritus-Ufer': 'Wilhelm-Spiritus-Ufer'
}
order = ['Kennedybrücke', 'Nordbrücke', 'Südbrücke', 'Estermannufer', 'Von-Sandt-Ufer', 'Rhenusallee', 
         'Brühler Straße', 'Wilhelm-Spiritus-Ufer', 'Mc Cloy Weg', 'Weg auf Damm Neil']
df.rename(columns=column_mapping, inplace=True)
df = df[order]
df = df.applymap(lambda x: -1 if pd.isna(x) else x).astype(int)
df = df.resample('H').sum()

df

,Kennedybrücke,Nordbrücke,Südbrücke,Estermannufer,Von-Sandt-Ufer,Rhenusallee,Brühler Straße,Wilhelm-Spiritus-Ufer,Mc Cloy Weg,Weg auf Damm Neil
Time,,,,,,,,,,
2019-01-01 00:00:00,48,13,18,1,6,4,8,17,3,2
2019-01-01 01:00:00,117,5,21,5,3,7,5,5,0,5
2019-01-01 02:00:00,115,13,17,0,4,9,12,11,2,12
2019-01-01 03:00:00,95,2,7,0,2,8,12,4,0,8
2019-01-01 04:00:00,62,0,7,0,4,3,3,13,2,3
...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,159,5,33,0,15,11,16,14,2,21
2019-12-31 20:00:00,92,6,22,0,3,7,2,8,5,6
2019-12-31 21:00:00,79,8,7,0,5,3,3,4,4,5


In [3]:
percentage_negative = ((df < 0).sum() / df.count()) * 100
percentage_negative

0
Kennedybrücke            0.0
Nordbrücke               0.0
Südbrücke                0.0
Estermannufer            0.0
Von-Sandt-Ufer           0.0
Rhenusallee              0.0
Brühler Straße           0.0
Wilhelm-Spiritus-Ufer    0.0
Mc Cloy Weg              0.0
Weg auf Damm Neil        0.0
dtype: float64

## Auszüge

In [25]:
kennedy2019 = df['Kennedybrücke']
bonn2019_tag = df.sum(axis=1).resample('D').sum()
bonn2019_monat = bonn2019_tag.resample('M').sum()
%store bonn2019_tag bonn2019_monat kennedy2019

Stored 'bonn2019_tag' (Series)
Stored 'bonn2019_monat' (Series)
Stored 'kennedy2019' (Series)


In [26]:
df.to_csv('df_Bonn2019.csv')
bonn2019_tag.to_csv('Bonn2019_täglich.csv')